<a href="https://colab.research.google.com/github/sasachichito/knowledge/blob/master/computer/%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86_%E8%A6%81%E7%B4%84_%E6%8A%BD%E8%B1%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 依存関係

In [ ]:
!pip install transformers[ja] deep-translator

from urllib.request import urlopen
from bs4 import BeautifulSoup

def webpage_to_text(url, selector):
  text = ''
  with urlopen(url) as res:
      html = res.read().decode('UTF-8', 'ignore')
      soup = BeautifulSoup(html, 'html.parser')
      # article = soup.find('div', class_="articleBody")
      # text = article.get_text(strip=True)
      article = soup.select(selector)
      text = ''
      for p in article:
        text += p.get_text(strip=True)
      return text

def trim_last_halfway_sentence(text, period_char):
  if text.endswith(period_char):
    return text

  last_period_index = text.rfind(period_char)
  return text[:last_period_index + 1]

# 抽象型要約

In [ ]:
from deep_translator import GoogleTranslator
from transformers import pipeline

def summary_text(text_ja, text_en):
  print('==============origin================')
  print(text_ja)
  print(text_en)
  print('====================================')

  text = text_en

  # 結果を日本語で確認するためにdeep_translatorを使用
  translator_for_print_ja = GoogleTranslator(source='auto', target="ja")

  print("\n" + '>>>>> google/pegasus-large' + "\n")
  pega_l_summarizer = pipeline("summarization", model="google/pegasus-large")
  summarized_pega_l = pega_l_summarizer(text, max_length=130, min_length=30, do_sample=False)
  print([item['summary_text'] for item in summarized_pega_l][0])
  print(translator_for_print_ja.translate([item['summary_text'] for item in summarized_pega_l][0]))

  print("\n" + '>>>>> google/pegasus-xsum' + "\n")
  pega_x_summarizer = pipeline("summarization", model="google/pegasus-xsum")
  summarized_pega_x = pega_x_summarizer(text, max_length=130, min_length=30, do_sample=False)
  print([item['summary_text'] for item in summarized_pega_x][0])
  print(translator_for_print_ja.translate([item['summary_text'] for item in summarized_pega_x][0]))

  print("\n" + '>>>>> facebook/bart-large-cnn' + "\n")
  bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
  summarized_bart = bart_summarizer(text, max_length=130, min_length=30, do_sample=False)
  print([item['summary_text'] for item in summarized_bart][0])
  print(translator_for_print_ja.translate([item['summary_text'] for item in summarized_bart][0]))

  print("\n" + '>>>>> csebuetnlp/mT5_multilingual_XLSum' + "\n")
  t5_summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")
  summarized_t5 = t5_summarizer(text, max_length=130, min_length=30, do_sample=True)
  print([item['summary_text'] for item in summarized_t5][0])
  print(translator_for_print_ja.translate([item['summary_text'] for item in summarized_t5][0]))

# 日本語記事の要約
# text_ja = webpage_to_text('https://xtech.nikkei.com/atcl/nxt/column/18/02828/050900001/', '.articleBody p')
# text_ja = webpage_to_text('https://xtech.nikkei.com/atcl/nxt/column/18/02783/051000018/', '.articleBody p')
text_ja = webpage_to_text('https://xtech.nikkei.com/atcl/nxt/column/18/02252/051400006/', '.articleBody p')
text_ja = trim_last_halfway_sentence(text_ja[:800], '。')
translator_to_en = GoogleTranslator(source='auto', target="en")
text_en = translator_to_en.translate(text_ja)
summary_text(text_ja, text_en)

# 英語記事の要約
text_en = webpage_to_text('https://edition.cnn.com/2024/05/13/politics/takeaways-michael-cohen-testimony-donald-trump-day-16/index.html', '.article__content p')
text_en = trim_last_halfway_sentence(text_en[:800], '.')
summary_text('', text_en)